<h1>Chapter 6: Data Loading, Storage, and File Formats<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#6.1-Reading-and-Writing-Data-in-Text-Format" data-toc-modified-id="6.1-Reading-and-Writing-Data-in-Text-Format-1">6.1 Reading and Writing Data in Text Format</a></span><ul class="toc-item"><li><span><a href="#Reading-Text-Files-in-Pieces" data-toc-modified-id="Reading-Text-Files-in-Pieces-1.1">Reading Text Files in Pieces</a></span></li><li><span><a href="#Writing-Data-to-Text-Format" data-toc-modified-id="Writing-Data-to-Text-Format-1.2">Writing Data to Text Format</a></span></li><li><span><a href="#Working-with-Other-Delimited-Formats" data-toc-modified-id="Working-with-Other-Delimited-Formats-1.3">Working with Other Delimited Formats</a></span></li><li><span><a href="#JSON-Data" data-toc-modified-id="JSON-Data-1.4">JSON Data</a></span></li><li><span><a href="#XML-and-HTML:-Web-Scraping" data-toc-modified-id="XML-and-HTML:-Web-Scraping-1.5">XML and HTML: Web Scraping</a></span><ul class="toc-item"><li><span><a href="#Parsing-XML-with-lxml.objectify" data-toc-modified-id="Parsing-XML-with-lxml.objectify-1.5.1">Parsing XML with lxml.objectify</a></span></li></ul></li></ul></li><li><span><a href="#6.2-Binary-Data-Formats" data-toc-modified-id="6.2-Binary-Data-Formats-2">6.2 Binary Data Formats</a></span><ul class="toc-item"><li><span><a href="#Reading-Microsoft-Excel-Files" data-toc-modified-id="Reading-Microsoft-Excel-Files-2.1">Reading Microsoft Excel Files</a></span></li><li><span><a href="#Using-HDF5-Format" data-toc-modified-id="Using-HDF5-Format-2.2">Using HDF5 Format</a></span></li></ul></li><li><span><a href="#6.3-Interacting-with-Web-APIs" data-toc-modified-id="6.3-Interacting-with-Web-APIs-3">6.3 Interacting with Web APIs</a></span></li><li><span><a href="#6.4-Interacting-with-Databases" data-toc-modified-id="6.4-Interacting-with-Databases-4">6.4 Interacting with Databases</a></span></li></ul></div>

In [1]:
# If you use Colab Notebook, you can uncomment the following to mount your Google Drive to Colab
# After that, your colab notebook can read/write files and data in your Google Drive

#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
# If you use Colab Notebook, please change the current directory to be the folder that you save 
# your Notebook and data folder for example, I save my Colab files and data in the following location

#%cd /content/drive/MyDrive/Colab\ Notebooks

In [3]:
# import required libraries and modules, and define default setting for the notebook

import numpy as np
np.random.seed(12345)

import pandas as pd # https://pandas.pydata.org/  Check the documentation there
from pandas import Series, DataFrame # import modules into the local namespace if they are frequently used

import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
pd.options.display.max_columns = 20
pd.options.display.max_colwidth = 80
np.set_printoptions(precision=4, suppress=True)

## 6.1 Reading and Writing Data in Text Format

Table 6-1. Text and binary data loading functions in pandas
Table 6-2 lists some frequently used options in pandas.read_csv

In [4]:
# cat shell command to print the raw contents of the file to the screen

!cat examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [5]:
# use pandas.read_csv to read it into a DataFrame

df = pd.read_csv("examples/ex1.csv")
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
# A file will not always have a header row. 
# Below is an example.

!cat examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [7]:
# When reading a csv without a header, we can allow pandas to assign default column names

pd.read_csv("examples/ex2.csv", header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
# When reading a csv without a header, we can also specify column names:

pd.read_csv("examples/ex2.csv", names=["a", "b", "c", "d", "message"])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
# we can also specify a column to be the index of the returned DataFrame
# in this example, we would like the column "message" to be the index column

names = ["a", "b", "c", "d", "message"]
pd.read_csv("examples/ex2.csv", names=names, index_col="message")

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [10]:
!cat examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [11]:
# we can form a hierarchical index from multiple columns by passing a list of column numbers or names:

parsed = pd.read_csv("examples/csv_mindex.csv",
                     index_col=["key1", "key2"])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [12]:
# a table might not have a fixed delimiter, using whitespace or some other pattern to separate fields

# in this example, whitespace are used to separate fields
!cat examples/ex3.txt

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


In [13]:
# we can pass a regular expression as a delimiter for pandas.read_csv. 
# This can be expressed by the regular expression \s+,

# there was one fewer column name than the number of data rows,
# pandas.read_csv infers that the first column should be the DataFrame’s index in this special case

result = pd.read_csv("examples/ex3.txt",sep="\s+") # if you delete sep='\s+', what would the result look like?
result 



,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [14]:
!cat examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [15]:
# The skiprows argument allows for skipping rows

# In this example, we would like to skip rows begin with symbol #

pd.read_csv("examples/ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [16]:
!cat examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [17]:
result = pd.read_csv("examples/ex5.csv")
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [18]:
# the isna method finds NaN

pd.isna(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [19]:
# na_values option accepts a sequence of strings to add to the default list of strings recognized as missing

result = pd.read_csv("examples/ex5.csv", na_values=["NULL"])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [20]:
# pandas.read_csv has a list of many default NA value representations, 
# but these defaults can be disabled with the keep_default_na option

result2 = pd.read_csv("examples/ex5.csv", keep_default_na=False)
result2


,something,a,b,c,d,message
0,one,1,2,3,4,NA
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [21]:
# in this example, when the option keep_default_na is set to be False, 
# the empty cell result2[1,'c'] is not treated as NaN
# the cell result2[0,'message'] is not treated as NaN, but a string 'NA'
# therefore, the test isna() returns False at any location of the DataFrame

result2.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False


In [22]:
# In this example, string "NA" is treated as NaN, which is claimed using the option na_values=["NA"]

result3 = pd.read_csv("examples/ex5.csv", keep_default_na=False,
                      na_values=["NA"])
result3

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [23]:
# therefore, when search NaN using isna() method, results3[0,'message'] is treated as NaN
result3.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,False,False,False
2,False,False,False,False,False,False


In [24]:
# Different NA sentinels can be specified for each column in a dictionary

sentinels = {"message": ["foo", "NA"], "something": ["two"]}
pd.read_csv("examples/ex5.csv", na_values=sentinels,
            keep_default_na=False)

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,NaN,5,6,,8,world
2,three,9,10,11,12,NaN


### Reading Text Files in Pieces

When processing very large files or figuring out the right set of arguments to correctly
process a large file, we may want to read only a small piece of a file or iterate
through smaller chunks of the file.

In [25]:
# we make the pandas display settings more compact by showing only 10 rows

pd.options.display.max_rows = 10

In [26]:
# ex6.csv is a file with 10,000 rows

result = pd.read_csv("examples/ex6.csv")
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [27]:
# nrows specifies how many rows to display

pd.read_csv("examples/ex6.csv", nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [28]:
# To read a file in pieces, specify a chunksize as a number of rows:

chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)
type(chunker)

# the dataset ex6.csv is divided into ten pieces with each containing 1000 rows

pandas.io.parsers.readers.TextFileReader

In [29]:
# The TextFileReader object returned by pandas.read_csv allows you to iterate over
# the parts of the file according to the chunksize. For example, we can iterate over
# ex6.csv, aggregating the value counts in the "key" column, like so.

chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)

tot = pd.Series([], dtype='int64') # create an empty Series with datatype int64
for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0) 
    
tot = tot.sort_values(ascending=False)

# for each piece, find the frequency by values in "key" using value_counts(). 
# If any value does not occur in that piece, set the frequency to be zero.
# iterate over pieces to find the cumulative counts.
# sort the frequency distribution in a descending order

In [30]:
# display the top 10 high-frequency 
tot[:10]

key
E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### Writing Data to Text Format

In [31]:
data = pd.read_csv("examples/ex5.csv")
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [32]:
# Using DataFrame’s to_csv method, we can write the data out to a comma-separated file

data.to_csv("examples/out.csv")
!cat examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [33]:
# Other delimiters can be used, of course (writing to sys.stdout, so it prints the text
# result to the console rather than a file):

import sys
data.to_csv(sys.stdout, sep="|")

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [34]:
# Missing values appear as empty strings in the output. You might want to denote them
# by some other sentinel value

data.to_csv(sys.stdout, na_rep="NULL")

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [35]:
# With no other options specified, both the row and column labels are written. 
# Both of these can be disabled:

data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [36]:
# You can also write only a subset of the columns, and in an order of your choosing

data.to_csv(sys.stdout, index=False, columns=["a", "b", "c"])

a,b,c
1,2,3.0
5,6,
9,10,11.0


### Working with Other Delimited Formats

It’s possible to load most forms of tabular data from disk using functions like pandas.read_csv. In some cases, however, some manual processing may be necessary.
It’s not uncommon to receive a file with one or more malformed lines that trip up
pandas.read_csv.

In [37]:
# this is an example that all elements are with double duote

!cat examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [38]:
# pass any open file or file-like object to csv.reader

import csv

f = open("examples/ex7.csv")
reader = csv.reader(f)

In [39]:
# Iterating through the reader like a file yields lists of values with any quote characters removed:

for line in reader:
    print(line)
f.close()

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [40]:
# From there, it’s up to you to do the wrangling necessary to put the data in the form that you need

# First, we read the file into a list of lines

with open("examples/ex7.csv") as f:
    lines = list(csv.reader(f))
    
lines

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3']]

In [41]:
#Then we split the lines into the header line and the data lines

header, values = lines[0], lines[1:]

In [42]:
# Then we can create a dictionary of data columns using a dictionary comprehension
# and the expression zip(*values) (beware that this will use a lot of memory on large
# files), which transposes rows to columns

data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

### JSON Data

JSON (short for JavaScript Object Notation) has become one of the standard formats
for sending data by HTTP request between web browsers and other applications. It
is a much more free-form data format than a tabular text form like CSV.

JSON is very nearly valid Python code with the exception of its null value null and
some other nuances (such as disallowing trailing commas at the end of lists). The
basic types are objects (dictionaries), arrays (lists), strings, numbers, Booleans, and
nulls. All of the keys in an object must be strings. There are several Python libraries
for reading and writing JSON data.

In [43]:
obj = """
{"name": "Wes",
 "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]},
              {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]
}
"""


In [44]:
# To convert a JSON string to Python form, use json.loads

import json
result = json.loads(obj)
result

# result is a dictionary

{'name': 'Wes',
 'cities_lived': ['Akron', 'Nashville', 'New York', 'San Francisco'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 34, 'hobbies': ['guitars', 'soccer']},
  {'name': 'Katie', 'age': 42, 'hobbies': ['diving', 'art']}]}

In [45]:
# json.dumps converts a Python object back to JSON

asjson = json.dumps(result)
asjson

'{"name": "Wes", "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"], "pet": null, "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]}, {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]}'

In [46]:
# How we convert a JSON object or list of objects to a DataFrame or some other data structure 
# for analysis will be up to us. 
# Conveniently, we can pass a list of dictionaries (which were previously JSON objects) to the DataFrame
# constructor and select a subset of the data fields:

siblings = pd.DataFrame(result["siblings"], columns=["name", "age"])
siblings

,name,age
0,Scott,34
1,Katie,42


In [47]:
!cat examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [48]:
# The pandas.read_json can automatically convert JSON datasets in specific arrangements
# into a Series or DataFrame
# The default options for pandas.read_json assume that each object in the JSON array is a row in the table

data = pd.read_json("examples/example.json")
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [49]:
# you need to export data from pandas to JSON, one way is to use the to_json
# methods on Series and DataFrame

# for DataFrame, the default orient for to_json is columns. 
# That is, each columns is converted to an item in the dictionary

data.to_json(sys.stdout) 


{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}

In [50]:
# orient="records": list like [{column -> value}, … , {column -> value}]
# if the option orient="records", each row is saved as an iten in the dictioinary, 
# and the column labels are keys 

data.to_json(sys.stdout, orient="records")

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]

### XML and HTML: Web Scraping

Python has many libraries for reading and writing data in the ubiquitous HTML and
XML formats. Examples include lxml, Beautiful Soup, and html5lib. While lxml is
comparatively much faster in general, the other libraries can better handle malformed
HTML or XML files.

In [51]:
!cat examples/fdic_failed_bank_list.html

<!DOCTYPE html>
<!-- saved from url=(0057)https://www.fdic.gov/bank/individual/failed/banklist.html -->
<html lang="en-US"><!-- Content language is American English. --><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
<title>FDIC: Failed Bank List</title>
<!-- Meta Tags -->
<meta charset="UTF-8">
<!-- Unicode character encoding -->
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<!-- Turns off IE Compatiblity Mode -->
<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1">
<!-- Makes it so phones don't auto zoom out. -->
<meta name="author" content="DRR">
<meta http-equiv="keywords" name="keywords" content="banks, financial institutions, failed, failure, closing, deposits, depositors, banking services, assuming institution, acquiring institution, claims">
<!-- CSS -->
<link rel="stylesheet" type="text/css" href="./fdic_failed_bank_list_files/responsive.css">
<link rel="stylesheet" type="text/css" href="./fdic_failed_bank_list

			  <td class="cert">29730</td>
			  <td class="ai">OneWest Bank, FSB</td>
			  <td class="closing">July 11, 2008</td>
			  <td class="updated">April 22, 2015</td>
			  </tr><tr name="hidethis" class="odd">
			  <td class="institution"><a href="https://www.fdic.gov/bank/individual/failed/first_integrity_bank.html">First Integrity Bank, NA</a></td>
			  <td class="city">Staples</td>
			  <td class="state">MN</td>
			  <td class="cert">12736</td>
			  <td class="ai">First International Bank and Trust</td>
			  <td class="closing">May 30, 2008</td>
			  <td class="updated">October 20, 2016</td>
			  </tr><tr name="hidethis" class="even">
			  <td class="institution"><a href="https://www.fdic.gov/bank/individual/failed/anb.html">ANB Financial, NA</a></td>
			  <td class="city">Bentonville</td>
			  <td class="state">AR</td>
			  <td class="cert">33901</td>
			  <td class="ai">Pulaski Bank and Trust Company</td>
			  <td class="closing">May 9, 2008</td>
			  <td class="updated">August 28, 

In [52]:
# use the pandas.read_html function to read html files

tables = pd.read_html("examples/fdic_failed_bank_list.html")
print(len(tables),'\n')
tables

1 



[                             Bank Name             City  ST   CERT  \
 0                          Allied Bank         Mulberry  AR     91   
 1         The Woodbury Banking Company         Woodbury  GA  11297   
 2               First CornerStone Bank  King of Prussia  PA  35312   
 3                   Trust Company Bank          Memphis  TN   9956   
 4           North Milwaukee State Bank        Milwaukee  WI  20364   
 ..                                 ...              ...  ..    ...   
 542                 Superior Bank, FSB         Hinsdale  IL  32646   
 543                Malta National Bank            Malta  OH   6629   
 544    First Alliance Bank & Trust Co.       Manchester  NH  34264   
 545  National State Bank of Metropolis       Metropolis  IL   3815   
 546                   Bank of Honolulu         Honolulu  HI  21029   
 
                    Acquiring Institution        Closing Date  \
 0                           Today's Bank  September 23, 2016   
 1              

In [53]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [54]:
# convert string to date time format using pandas.to_datetime function

close_timestamps = pd.to_datetime(failures["Closing Date"])

In [55]:
# use .dt to extract date time like information such as year, month, day, hour, minute, and second
# use value_counts() method to calculate the frequency of cases by year

close_timestamps.dt.year.value_counts()

Closing Date
2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: count, Length: 15, dtype: int64

#### Parsing XML with lxml.objectify

XML is another common structured data format supporting hierarchical, nested data
with metadata. XML and HTML are structurally similar, but XML is more general.

In [56]:
# # For many years, the New York Metropolitan Transportation Authority (MTA) published
# a number of data series about its bus and train services in XML format. Here
# we’ll look at the performance data, which is contained in a set of XML files. Each train
# or bus service has a different file

!cat datasets/mta_perf/Performance_MNR.xml

<?xml  version="1.0" encoding="ISO-8859-1"?>
<PERFORMANCE>
<INDICATOR>
  <INDICATOR_SEQ>28445</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>On-Time Performance (West of Hudson)</INDICATOR_NAME>
  <DESCRIPTION>Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate service on these lines.
</DESCRIPTION>
  <PERIOD_YEAR>2008</PERIOD_YEAR>
  <PERIOD_MONTH>1</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>95.00</YTD_TARGET>
  <YTD_ACTUAL>96.90</YTD_ACTUAL>
  <MONTHLY_TARGET>95.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL>96.90</MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICAT

  <INDICATOR_SEQ>55512</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Total Ridership</INDICATOR_NAME>
  <DESCRIPTION>The number of passengers from whom the agency receives a fare (cash, train tickets, time-based passes, etc.) Ridership data is preliminary and subject to revision as well as adjustments warranted by annual audit review.</DESCRIPTION>
  <PERIOD_YEAR>2008</PERIOD_YEAR>
  <PERIOD_MONTH>12</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>-</INDICATOR_UNIT>
  <DECIMAL_PLACES>0</DECIMAL_PLACES>
  <YTD_TARGET>81,739,667.00</YTD_TARGET>
  <YTD_ACTUAL>83,555,228.00</YTD_ACTUAL>
  <MONTHLY_TARGET>7,323,974.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL>7,165,229.00</MONTHLY_ACTUAL>
</INDICATOR>
<INDICATOR>
  <INDICATOR_SEQ>55512</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NA

In [57]:
# Using lxml.objectify, we parse the file and get a reference to the root node of the XML file with getroot:
    
from lxml import objectify

path = "datasets/mta_perf/Performance_MNR.xml"
with open(path) as f:
    parsed = objectify.parse(f)
root = parsed.getroot()

In [58]:
# root.INDICATOR returns a generator yielding each <INDICATOR> XML element. 
# For each record, we can populate a dictionary of tag names (like YTD_ACTUAL) 
# to datavalues (excluding a few tags) by running the following code:
    
data = []


skip_fields = ["PARENT_SEQ", "INDICATOR_SEQ",
               "DESIRED_CHANGE", "DECIMAL_PLACES"]

for elt in root.INDICATOR: # iterate over each INDICATOR
    el_data = {}
    for child in elt.getchildren(): # getchildren() returns a field with associated value. e.g., <CATEGORY>Service Indicators</CATEGORY>
        if child.tag in skip_fields: # child.tag is the field name, e.g., CATEGORY
            continue
        el_data[child.tag] = child.pyval # child.pyval is the field value. e.g., Service Indicators
    data.append(el_data) # us the append method to add the parsed elements of each INDICATOR as a new row to data

In [59]:
# convert the dictionary into a DataFrame
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 minute...,2008,1,Service Indicators,M,%,95.0,96.9,95.0,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 minute...,2008,2,Service Indicators,M,%,95.0,96.0,95.0,95.0
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 minute...,2008,3,Service Indicators,M,%,95.0,96.3,95.0,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 minute...,2008,4,Service Indicators,M,%,95.0,96.8,95.0,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 minute...,2008,5,Service Indicators,M,%,95.0,96.6,95.0,95.8


In [60]:
# pandas’s pandas.read_xml function turns the above process into a one-line expression :)

perf2 = pd.read_xml(path)
perf2.head()

,INDICATOR_SEQ,PARENT_SEQ,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,DESIRED_CHANGE,INDICATOR_UNIT,DECIMAL_PLACES,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 minute...,2008,1,Service Indicators,M,U,%,1,95.00,96.90,95.00,96.90
1,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 minute...,2008,2,Service Indicators,M,U,%,1,95.00,96.00,95.00,95.00
2,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 minute...,2008,3,Service Indicators,M,U,%,1,95.00,96.30,95.00,96.90
3,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 minute...,2008,4,Service Indicators,M,U,%,1,95.00,96.80,95.00,98.30
4,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 minute...,2008,5,Service Indicators,M,U,%,1,95.00,96.60,95.00,95.80


## 6.2 Binary Data Formats

One simple way to store (or serialize) data in binary format is using Python’s built-in
pickle module. pandas objects all have a to_pickle method that writes the data to
disk in pickle format

In [61]:
# the to_pickle method write the data into disk in pickle format.

frame = pd.read_csv("examples/ex1.csv")
frame
frame.to_pickle("examples/frame_pickle")

In [62]:
# the read_pickle method read data in pickle format from the disk.

pd.read_pickle("examples/frame_pickle")

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [63]:
# !rm removes files from the disk

!rm examples/frame_pickle

In [64]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [65]:
# pandas has built-in support for several other open source binary data formats, such
# as HDF5, ORC, and Apache Parquet. For example, if you install the pyarrow package
# (conda install pyarrow), then you can read Parquet files with pandas.read_parquet

# install the pyarrow package if it's not installed before


fec = pd.read_parquet('datasets/fec/fec.parquet')
fec

,cmte_id,cand_id,cand_nm,contbr_nm,contbr_city,contbr_st,contbr_zip,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt,receipt_desc,memo_cd,memo_text,form_tp,file_num
0,C00410118,P20002978,"Bachmann, Michelle","HARVEY, WILLIAM",MOBILE,AL,366010290,RETIRED,RETIRED,250.0,20-JUN-11,None,None,None,SA17A,736166
1,C00410118,P20002978,"Bachmann, Michelle","HARVEY, WILLIAM",MOBILE,AL,366010290,RETIRED,RETIRED,50.0,23-JUN-11,None,None,None,SA17A,736166
2,C00410118,P20002978,"Bachmann, Michelle","SMITH, LANIER",LANETT,AL,368633403,INFORMATION REQUESTED,INFORMATION REQUESTED,250.0,05-JUL-11,None,None,None,SA17A,749073
3,C00410118,P20002978,"Bachmann, Michelle","BLEVINS, DARONDA",PIGGOTT,AR,724548253,NONE,RETIRED,250.0,01-AUG-11,None,None,None,SA17A,749073
4,C00410118,P20002978,"Bachmann, Michelle","WARDENBURG, HAROLD",HOT SPRINGS NATION,AR,719016467,NONE,RETIRED,300.0,20-JUN-11,None,None,None,SA17A,736166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001726,C00500587,P20003281,"Perry, Rick","GORMAN, CHRIS D. MR.",INFO REQUESTED,XX,99999,INFORMATION REQUESTED PER BEST EFFORTS,INFORMATION REQUESTED PER BEST EFFORTS,5000.0,29-SEP-11,REATTRIBUTION / REDESIGNATION REQUESTED (AUTOMATIC),None,REATTRIBUTION / REDESIGNATION REQUESTED (AUTOMATIC),SA17A,751678
1001727,C00500587,P20003281,"Perry, Rick","DUFFY, DAVID A. MR.",INFO REQUESTED,XX,99999,DUFFY EQUIPMENT COMPANY INC.,BUSINESS OWNER,2500.0,30-SEP-11,None,None,None,SA17A,751678
1001728,C00500587,P20003281,"Perry, Rick","GRANE, BRYAN F. MR.",INFO REQUESTED,XX,99999,INFORMATION REQUESTED PER BEST EFFORTS,INFORMATION REQUESTED PER BEST EFFORTS,500.0,29-SEP-11,None,None,None,SA17A,751678
1001729,C00500587,P20003281,"Perry, Rick","TOLBERT, DARYL MR.",INFO REQUESTED,XX,99999,T.A.C.C.,LONGWALL MAINTENANCE FOREMAN,500.0,30-SEP-11,None,None,None,SA17A,751678


### Reading Microsoft Excel Files

pandas also supports reading tabular data stored in Excel 2003 (and higher) files
using either the pandas.ExcelFile class or pandas.read_excel function. Internally,
these tools use the add-on packages xlrd and openpyxl to read old-style XLS and
newer XLSX files, respectively. These must be installed separately from pandas using
pip or conda.

In [66]:
pip install openpyxl xlrd

Note: you may need to restart the kernel to use updated packages.


In [67]:
# use the ExcelFile function to create an instance by passing a path to an xls or xlsx file:
xlsx = pd.ExcelFile("examples/ex1.xlsx")

In [68]:
# the sheet_names method can show the list of available sheet names
xlsx.sheet_names

['Sheet1']

In [69]:
# the parse method can read the data in a sheet into DataFrame

xlsx.parse(sheet_name="Sheet1")

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [70]:
# This Excel table has an index column, so we can indicate that with the index_col argument

xlsx.parse(sheet_name="Sheet1", index_col=0)

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [71]:
# If we are reading multiple sheets in a file, then it is faster to create the pandas.Excel File, 
# but we can also simply pass the filename to pandas.read_excel:

frame = pd.read_excel("examples/ex1.xlsx", sheet_name="Sheet1")
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [72]:
pip install 'xlsxwriter>=1.4.3'

Note: you may need to restart the kernel to use updated packages.


In [73]:
# To write pandas data to Excel format, we must first create an ExcelWriter, 
# then write data to it using the pandas object’s to_excel method

writer = pd.ExcelWriter("examples/ex2.xlsx")
frame.to_excel(writer, "Sheet1")
writer.close()

In [74]:
# We can also pass a file path to to_excel and avoid the ExcelWriter

frame.to_excel("examples/ex2.xlsx")

In [75]:
!rm examples/ex2.xlsx

### Using HDF5 Format

HDF5 is a respected file format intended for storing large quantities of scientific array
data. It is available as a C library, and it has interfaces available in many other languages,
including Java, Julia, MATLAB, and Python. The “HDF” in HDF5 stands for
hierarchical data format. Each HDF5 file can store multiple datasets and supporting
metadata. Compared with simpler formats, HDF5 supports on-the-fly compression
with a variety of compression modes, enabling data with repeated patterns to be
stored more efficiently. HDF5 can be a good choice for working with datasets that
don’t fit into memory, as you can efficiently read and write small sections of much
larger arrays.

In [76]:
# To get started with HDF5 and pandas, you must first install PyTables by installing the
# tables package: 
#conda install pytables

# if we use pip, then installing tables:
#pip install tables

In [77]:
pip install tables

Note: you may need to restart the kernel to use updated packages.


In [78]:
# remove mydata.h5 if there is one in the specified directory:

!rm -f examples/mydata.h5

In [79]:
# pandas provides a high-level interface that simplifies storing Series and
# DataFrame objects. The HDFStore class works like a dictionary and handles the
# low-level details:

frame = pd.DataFrame({"a": np.random.standard_normal(100)}) # frame is a dictionary 
store = pd.HDFStore("examples/mydata.h5") # create a .h5 file
store["obj1"] = frame # save the dictionary frame as an object in the .h5 file
store["obj1_col"] = frame["a"] # the col name is "a"
store

<class 'pandas.io.pytables.HDFStore'>
File path: examples/mydata.h5

In [80]:
store["obj1"]

,a
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781
...,...
95,0.795253
96,0.118110
97,-0.748532
98,0.584970


In [81]:
# HDFStore supports two storage schemas, "fixed" and "table" (the default is "fixed"). 
# The latter is generally slower, but it supports query operations using a special syntax

store.put("obj2", frame, format="table") # HDFStore.put is a method to store DataFrame or Series into HDF5 files
# The put is an explicit version of the store["obj2"] = frame method but allows us to
# set other options like the storage format.

store.select("obj2", where=["index >= 10 and index <= 15"])
store.close()  # HDFStore.close is a method used in the pandas library to close an open HDFStore object

In [82]:
# The pandas.read_hdf function gives us a shortcut to these tools

frame.to_hdf("examples/mydata.h5", "obj3", format="table")
pd.read_hdf("examples/mydata.h5", "obj3",where=["index < 5"])

,a
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781


In [83]:
# we can delete the HDF5 file we created

import os
os.remove("examples/mydata.h5")

## 6.3 Interacting with Web APIs


Many websites have public APIs providing data feeds via JSON or some other format. # There are a number of ways to access these APIs from Python; one method that we can use # is the requests package

In [84]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [85]:
# To find the last 30 GitHub issues for pandas on GitHub, we can make a GET HTTP
# request using the add-on requests library

# It’s a good practice to always call raise_for_status after using requests.get to check for HTTP errors.

import requests
url = "https://api.github.com/repos/pandas-dev/pandas/issues"
resp = requests.get(url)
resp.raise_for_status() # response.raise_for_status() returns an HTTPError object if an error has occurred during the process
resp

<Response [200]>

In [86]:
# The response object’s json method will return a Python object containing the parsed
# JSON data as a dictionary or list (depending on what JSON is returned):
    
data = resp.json()
data

[{'url': 'https://api.github.com/repos/pandas-dev/pandas/issues/57295',
  'repository_url': 'https://api.github.com/repos/pandas-dev/pandas',
  'labels_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/57295/labels{/name}',
  'comments_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/57295/comments',
  'events_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/57295/events',
  'html_url': 'https://github.com/pandas-dev/pandas/pull/57295',
  'id': 2123540416,
  'node_id': 'PR_kwDOAA0YD85mSg8X',
  'number': 57295,
  'title': 'BUG: Updated _pprint_seq to use enumerate instead of next',
  'user': {'login': 'cliffckerr',
   'id': 3239256,
   'node_id': 'MDQ6VXNlcjMyMzkyNTY=',
   'avatar_url': 'https://avatars.githubusercontent.com/u/3239256?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/cliffckerr',
   'html_url': 'https://github.com/cliffckerr',
   'followers_url': 'https://api.github.com/users/cliffckerr/followers',
   'following_url':

In [87]:
data[0]["title"]

'BUG: Updated _pprint_seq to use enumerate instead of next'

In [88]:
# Each element in data is a dictionary containing all of the data found on a GitHub
# issue page (except for the comments). We can pass data directly to pandas.DataFrame 
# and extract fields of interest

# In this example, we are interested in four fields: "number", "title", "labels", and "state"
issues = pd.DataFrame(data, columns=["number", "title",
                                     "labels", "state"])
issues

,number,title,labels,state
0,57295,BUG: Updated _pprint_seq to use enumerate instead of next,[],open
1,57294,Backport PR #57233 on branch 2.2.x (REGR: Fix to_numpy conversion for arrow ...,"[{'id': 3303158446, 'node_id': 'MDU6TGFiZWwzMzAzMTU4NDQ2', 'url': 'https://a...",open
2,57293,BUG: `sort_index` not preserving `index` when `ascending=False`,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ==', 'url': 'https://api.github...",open
3,57291,BUG: pd.merge has unexpected behaviour when joining on index / column,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ==', 'url': 'https://api.github...",open
4,57290,DOC: CoW User Guide - Previous behavior is current,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OTk=', 'url': 'https://api.githu...",open
...,...,...,...,...
25,57254,CoW: Enforce some deprecations on the datafame level,"[{'id': 2085877452, 'node_id': 'MDU6TGFiZWwyMDg1ODc3NDUy', 'url': 'https://a...",open
26,57252,PERF: RangeIndex.append with the same index,"[{'id': 8935311, 'node_id': 'MDU6TGFiZWw4OTM1MzEx', 'url': 'https://api.gith...",open
27,57250,REGR/DOC: pd.concat should special case DatetimeIndex to sort even when sort...,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OTk=', 'url': 'https://api.githu...",open
28,57245,DOC: Pandas Case_When,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OTk=', 'url': 'https://api.githu...",open


## 6.4 Interacting with Databases

In a business setting, a lot of data may not be stored in text or Excel files. SQL-based
relational databases (such as SQL Server, PostgreSQL, and MySQL) are in wide use,
and many alternative databases have become quite popular. The choice of database
is usually dependent on the performance, data integrity, and scalability needs of an
application.

In [89]:
# pandas has some functions to simplify loading the results of a SQL query into a
# DataFrame. As an example, we create a SQLite3 database using Python’s built-in
# sqlite3 driver:
    
    
import sqlite3

# remove mydata.sqlite if it exists
!rm mydata.sqlite

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

con = sqlite3.connect("mydata.sqlite")
con.execute(query)
con.commit()

rm: mydata.sqlite: No such file or directory


In [90]:
data = [("Atlanta", "Georgia", 1.25, 6),
        ("Tallahassee", "Florida", 2.6, 3),
        ("Sacramento", "California", 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

In [91]:
cursor = con.execute("SELECT * FROM test")
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [92]:
cursor.description
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [93]:
import sqlalchemy as sqla
db = sqla.create_engine("sqlite:///mydata.sqlite")
pd.read_sql("SELECT * FROM test", db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [94]:
!rm mydata.sqlite